Functions

In [ ]:
import glob
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pathlib import Path
import itertools

def search(base_dir, wildcard):
    search_path = Path(base_dir) / wildcard
    files = glob.glob(str(search_path))

    if not files:
        raise FileNotFoundError(f"No files were found in: {search_path}")

    return files

def search_denoise_dir(
    experiment_id, 
    mri_id, 
    proc_id, 
    denoise_id, 
    sub_id, 
    ses_id, 
    task_id, 
    run_id,
):

    nifti = {}
    for descriptor in ["windowed", "denoised"]:
        wildcard = f"{experiment_id}/{mri_id}/derivatives/run_level_proc-{proc_id}_NIFTI/{denoise_id}/sub-{sub_id}/ses-{ses_id}/task-{task_id}/run-{run_id}/GLM/*desc-{descriptor}*bold.nii.gz"
        _nifti = search("/scratch", wildcard)
        assert len(_nifti) == 1
        nifti[descriptor] = _nifti[0]
    
    return nifti

def rename_nifti_to_metric(nifti_name, metric_name, search_frequency):

    return Path(nifti_name.replace('bold.', f"f-{search_frequency}_metric-{metric_name}."))

def create_directory(directory_path):
    path = Path(directory_path)
    if not path.exists():
        path.mkdir(parents=True)

Compute run average

In [ ]:
import nibabel as nib
import numpy as np

proc_ids = ["NONORDIC"] # ["NONORDIC", "NORDIC"]
dm_ids = ['00_experiment-min+motion24+wmcsfmean',]
descs = ['denoised','windowed']

for proc_id, dm_id, desc in itertools.product(proc_ids, dm_ids, descs):

    print(proc_id, dm_id, desc)
    # Get dtseries
    inputs = {
        "experiment_id": "1_attention",
        "mri_id": "7T",
        "proc_id": proc_id,
        "denoise_id": dm_id,
        "sub_id": "000",
        "ses_id": "20230801", 
        "task_id": "wbpilot",
        "run_id": '',
    }


    experiment_id = inputs["experiment_id"]
    mri_id = inputs["mri_id"]
    proc_id = inputs["proc_id"]
    denoise_id = inputs["denoise_id"]
    sub_id = inputs["sub_id"]
    ses_id = inputs["ses_id"]
    task_id = inputs["task_id"]
    
    niftis = !ls /scratch/{experiment_id}/{mri_id}/derivatives/run_level_proc-{proc_id}_NIFTI/{denoise_id}/sub-{sub_id}/ses-{ses_id}/task-{task_id}/run-??/GLM/*desc-{desc}*bold.nii.gz
    print(len(niftis), niftis[0], niftis[-1])
    
    """
    for i in dtseries:
        print(i)
    """

    run_avg_dir = Path(f"/scratch/{experiment_id}/{mri_id}/derivatives/run_level_proc-{proc_id}_NIFTI/{denoise_id}/sub-{sub_id}/ses-{ses_id}/task-{task_id}/run-avg/GLM")
    create_directory(run_avg_dir)
    # Average all runs
    bold_str_base = f"sub-{sub_id}_ses-{ses_id}_task-{task_id}_run-avg_desc-{desc}"
    out_nifti = f"{run_avg_dir}/{bold_str_base}_bold.nii.gz"
    for ix, nifti in enumerate(niftis):
        if ix == 0:
            !cp {nifti} {out_nifti}
        else:
            !fslmaths {out_nifti} -add {nifti} {out_nifti}
    !fslmaths {out_nifti} -div {len(niftis)} {out_nifti}


In [ ]:
niftis

Fit frequencies onto collapsed wholebrain data and compute R2 model fit

In [ ]:
METRIC_NAME = 'r2'
TR = 3.024
search_frequencies = [.2,.125]
proc_ids = ["NONORDIC"] # ["NONORDIC", "NORDIC"]
dm_ids = ['00_experiment-min+motion24+wmcsf_mean+scrub',]
run_ids = ['avg'] # '01', '03', '04', '05', '06', '07', '08', '09'

for proc_id, dm_id, run_id, search_frequency in itertools.product(proc_ids, dm_ids, run_ids, search_frequencies):

    # Get dtseries
    inputs = {
        "experiment_id": "1_attention",
        "mri_id": "7T",
        "proc_id": proc_id,
        "denoise_id": dm_id,
        "sub_id": "000",
        "ses_id": "20230801", 
        "task_id": "wbpilot",
        "run_id": run_id,
    }
    niftis = search_denoise_dir(**inputs)

    # Run on windowed and denoised data
    for descriptor in ["denoised", "windowed"]:

        _nifti = niftis[descriptor]
        metric_out = rename_nifti_to_metric(_nifti, METRIC_NAME, search_frequency)
        if metric_out.exists():
            print(f"{metric_out.stem} already generated.\nSkipping.")
            continue
        else:
            print(f"Generating metric: {metric_out.stem}")
        
        # Load dtseries
        img = nib.load(_nifti)
        ts_data = img.get_fdata()
        x, y, z, n_tps = ts_data.shape
    
        # Run GLM on each vertex
        r2_data = np.zeros((x,y,z))
        voxel_coordinates = itertools.product(range(x), range(y), range(z))

        n_voxels = np.prod((x,y,z))
        # Iterate over the voxel coordinates
        for voxel_idx, coordinates in enumerate(voxel_coordinates):
            # Process the voxel coordinates (x, y, z) here
            _x, _y, _z = coordinates
    
            if voxel_idx % 50_000 == 0:
                print(f"[PROGRESS] {str(voxel_idx).zfill(7)}/{str(n_voxels).zfill(7)}")
            
            # Get timeseries from a voxel, and associated timepoints
            y = ts_data[_x, _y, _z, :]
            # Skip if all values is zero
            if np.all(y == 0):
                continue
                
            t = np.linspace(0, TR*n_tps, n_tps+1)[:-1] # Non-phased timepoints
            t = np.fmod(t, 1/search_frequency) # Phased timepoints
            
            # GLM - fit phased
            X = np.vstack((np.sin(2*np.pi*t*search_frequency), np.cos(2*np.pi*t*search_frequency))).T
            X = sm.add_constant(X)
            model = sm.GLM(y, X, family=sm.families.Gaussian())
            try:
                result = model.fit()
                # Calculate R2
                y_pred = result.predict(X)
                y_mean = np.mean(y)
                ss_total = np.sum((y - y_mean) ** 2)  # Total sum of squares
                ss_residual = np.sum((y - y_pred) ** 2)  # Residual sum of squares
                r2 = 1 - (ss_residual / ss_total)  # R-squared
                # store r2
                r2_data[_x, _y, _z] = r2
            except:
                r2_data[_x, _y, _z] = -1
    
        # Save as dscalar
        r2_img = nib.Nifti1Image(r2_data, header = img.header, affine = img.affine)
        nib.save(r2_img, metric_out)